In [1]:
import ipywidgets as widgets
import pickle
from IPython.display import display, clear_output
from elo_util import add_player, elo_update, print_elos, reset_df
import matplotlib.pyplot as plt

In [2]:
class home_screen:
    def __init__(self):
        self.output = widgets.HBox()

        self.elo_display = widgets.Output()
        

        self.record_game = widgets.Button(
                            description='Record Game',
                            disabled=False,
                            button_style='', # 'success', 'info', 'warning', 'danger' or ''
                            tooltip='Record Game',)

        self.add_player = widgets.Button(
                            description='Add Player',
                            disabled=False,
                            button_style='', # 'success', 'info', 'warning', 'danger' or ''
                            tooltip='Add Player',)

        self.plot_elos = widgets.Button(
                            description='Plot ELOs',
                            disabled=False,
                            button_style='',
                            tooltipl='Plot ELOs'
        )
        self.reset = widgets.Button(
                            description='Reset Scores',
                            disabled=False,
                            button_style='', # 'success', 'info', 'warning', 'danger' or ''
                            tooltip='Reset Scores',)

    def handle_add_player(self,_):

        new_player = widgets.Text(
            placeholder='New player name',
            description='New player',
            disabled=False)

        def submit(_):
            new_player_name = new_player.value
            add_player(new_player_name)
            self.home()

        new_player.on_submit(submit)

        clear_output()
        self.output.children = [new_player]
        display(self.output)
    
    def handle_record_game(self,_):

        elo_df = pickle.load(open('elo_df.pkl','rb'))

        player1 = widgets.Select(
                        placeholder='Player 1',
                        options=elo_df.columns.values,
                        description='Player 1',
                        ensure_option=True,
                        disabled=False
                    )

        player2 = widgets.Select(
                        placeholder='Player 2',
                        options=elo_df.columns.values,
                        description='Player 2',
                        ensure_option=True,
                        disabled=False
                    )

        outcome = widgets.RadioButtons(
                    options=['Player 1 Win', 'Player 2 Win', 'Draw'],
                    description='Game Outcome',
                    disabled=False
                )

        confirm = widgets.Button(
                            description='Comfirm',
                            disabled=False,
                            button_style='', # 'success', 'info', 'warning', 'danger' or ''
                            tooltip='Confirm',)


        def handle_confirm(_):
            player_1 = player1.value
            player_2 = player2.value

            if player_1 == player_2:
                raise Exception
                #self.handle_record_game(_)

            if outcome.value == 'Player 1 Win':
                p1_score = 1
                p2_score = 0
            elif outcome.value == 'Player 2 Win':
                p1_score = 0
                p2_score = 1
            elif outcome.value == 'Draw':
                p1_score = 0.5
                p2_score = 0.5

            elo_update(player_1, player_2, p1_score, p2_score)
            self.home()

        confirm.on_click(handle_confirm)

        player_select = widgets.HBox([player1,player2])

        options = widgets.VBox([player_select, outcome, confirm])

        clear_output()
        self.output.children = [options]
        display(self.output)

    def handle_reset(self,_):

        question = widgets.Output()
        with question:
            print("Are you sure you want to reset scores and players?!")

        y = widgets.Button(
                            description='Yes',
                            disabled=False,
                            button_style='', # 'success', 'info', 'warning', 'danger' or ''
                            tooltip='Yes',)
        n = widgets.Button(
                            description='No',
                            disabled=False,
                            button_style='', # 'success', 'info', 'warning', 'danger' or ''
                            tooltip='No',)

        def handle_y(_):
            reset_df()
            self.home()

        def handle_n(_):
            self.home()

        y.on_click(handle_y)
        n.on_click(handle_n)

        clear_output()
        self.output.children = [widgets.VBox([question,widgets.HBox([y,n])])]
        display(self.output)

    def handle_plot_elos(self,_):
        elo_df =  pickle.load(open('elo_df.pkl','rb'))

        graph = widgets.Output()
        with graph:
            elo_df.plot.line()
            plt.xlabel('Time')
            plt.ylabel('ELO')

        close_plot = widgets.Button(
                            description='Close Plot',
                            disabled=False,
                            button_style='', # 'success', 'info', 'warning', 'danger' or ''
                            tooltip='Close Plot',)

        def handle_close_plot(_):
            clear_output()
            self.home()

        close_plot.on_click(handle_close_plot)

        clear_output()
        self.output.children = [widgets.VBox([graph,close_plot])]
        display(self.output)


    def home(self):
        clear_output()
        with self.elo_display:
            clear_output()
            print_elos()

        self.add_player.on_click(self.handle_add_player)
        self.record_game.on_click(self.handle_record_game)
        self.plot_elos.on_click(self.handle_plot_elos)
        self.reset.on_click(self.handle_reset)

        menu_buttons = widgets.HBox([self.record_game, self.add_player, self.plot_elos,self.reset])

        full = widgets.VBox([self.elo_display,menu_buttons])

        self.output.children = [full]
        display(self.output)

In [3]:
h = home_screen()
h.home()